- GNN つかう

In [1]:
import os

is_kaggle_notebook = os.path.exists("/kaggle/input")

# 必要パッケージをインストール
if is_kaggle_notebook:
    !pip install /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl

In [2]:
import os
import warnings
import sys
from pathlib import Path

import lightgbm as lgb
import numpy as np
import pandas as pd
from rdkit import rdBase

import torch
from torch.utils.data import Subset
from torch_geometric.loader import DataLoader

from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, mean_absolute_error


rdBase.DisableLog('rdApp.warning')
warnings.filterwarnings("ignore")

/home/kouya-takahashi/kaggle/opp2025/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
pr_number = 1

In [4]:
if is_kaggle_notebook:
    module_path = f"/kaggle/input/myproject-pr-{pr_number:04}"
    !mkdir src
    !cp -r $module_path/* src/
    src_path = "./"
else:
    src_path = "../"

sys.path.append(src_path)

from src.data import load_data, add_descriptors, add_external_data
from src.model import MoleculeDataset, GNNModel, smiles_to_graph, ATOM_MAP

In [5]:
exp = "exp014"
debug = False
n_splits = 5 if not debug else 2
targets = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']

num_epochs = 10
batch_size = 32
hidden_channels = 256

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

ATOM_LIST = ['C', 'O', 'N', 'H', 'F', 'Cl', 'Br', 'I', 'S', 'P', 'UNK']
ATOM_MAP = {atom: idx for idx, atom in enumerate(ATOM_LIST)}  # One-hotの位置

In [ ]:
# ---------------------------
# メイン処理
# ---------------------------
if debug:
    output_path = Path("../tmp") / exp
else:
    output_path = Path("../outputs") / exp

if is_kaggle_notebook:
    # kaggle notebook
    data_dir = Path("/kaggle/input")
    debug = False
else:
    # local
    data_dir = Path("../data/raw")

train, test = load_data(data_dir)
train = add_external_data(train, data_dir / "neurips-open-polymer-prediction-2025/train_supplement/dataset3.csv", "Tg")
train = add_external_data(train, data_dir / "neurips-open-polymer-prediction-2025/train_supplement/dataset1.csv", "Tc", {"TC_mean": "Tc"})
train = add_external_data(train, data_dir / "neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv", "FFV")

train["id"] = np.arange(len(train))
submission = pd.read_csv(data_dir / "neurips-open-polymer-prediction-2025/sample_submission.csv")

oof_dfs = []

for idx, target in enumerate(targets):
    print(f"============ {target} ============")
    df_train = train[train[target].notnull()]
    dataset = MoleculeDataset(df_train["SMILES"].tolist(), df_train[target].tolist())

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    fold_results = []
    oof = np.zeros(len(df_train))

    for fold, (train_idx, val_idx) in enumerate(kf.split(dataset)):
        print(f"\n=== Fold {fold + 1} / {n_splits} ===")
        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)
        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=True)

        num_node_features = dataset[0].num_node_features
        model = GNNModel(num_node_features, hidden_channels).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
        loss_fn = torch.nn.MSELoss()

        for epoch in range(1, num_epochs + 1):
            model.train()
            total_loss = 0
            for batch in train_loader:
                batch = batch.to(device)
                optimizer.zero_grad()
                pred = model(batch)
                loss = loss_fn(pred.squeeze(), batch.y.squeeze())
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            if epoch % 10 == 0:
                print(f"Epoch {epoch:2d} | Train Loss: {total_loss / len(train_loader):.4f}")
    
        model.eval()
        preds, trues = [], []
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                pred = model(batch).squeeze().cpu().numpy()
                true = batch.y.squeeze().cpu().numpy()
                preds.extend(pred.tolist())
                trues.extend(true.tolist())
        oof[val_idx] = preds
        
        mse = mean_squared_error(trues, preds)
        mae = mean_absolute_error(trues, preds)
        fold_results.append({"mse": mse, "mae": mae})
        print(f"Fold {fold+1} MSE: {mse:.4f} | MAE: {mae:.4f}")

    oof_df = pd.DataFrame({
        "id": df_train["id"],
        target: oof
    })
    oof_dfs.append(oof_df)

============ Tg ============

=== Fold 1 / 5 ===
Epoch 10 | Train Loss: 6819.5248
Fold 1 MSE: 7316.8083 | MAE: 63.9605

=== Fold 2 / 5 ===
Epoch 10 | Train Loss: 7229.0726
Fold 2 MSE: 7266.1044 | MAE: 64.1831

=== Fold 3 / 5 ===
Epoch 10 | Train Loss: 7562.2218
Fold 3 MSE: 7250.4118 | MAE: 70.0220

=== Fold 4 / 5 ===
Epoch 10 | Train Loss: 7229.0449
Fold 4 MSE: 6088.0806 | MAE: 63.1160

=== Fold 5 / 5 ===
Epoch 10 | Train Loss: 6763.6032
Fold 5 MSE: 7288.3113 | MAE: 69.2448
============ FFV ============

=== Fold 1 / 5 ===


In [ ]:
if is_kaggle_notebook:
    submission.to_csv("submission.csv", index=False)
    print("✅ submission saved to submission.csv")
else:
    # CV 計算
    oof_df = pd.DataFrame()
    oof_df["id"] = train["id"]
    for i_oof in oof_dfs:
        oof_df = oof_df.merge(i_oof, on="id", how="left")
    solution = train[["id"] + targets].copy()
    solution = solution.fillna(NULL_FOR_SUBMISSION)
    oof_df = oof_df.fillna(NULL_FOR_SUBMISSION)
    
    # 評価
    final_score = score(
        solution=solution,
        submission=oof_df,
    )
    print(f"\n📊 Final OOF Score (wMAE): {final_score:.6f}")
